In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import datetime as dt
from autogen import AssistantAgent
from inswitch.llm.model import get_openai_model_config
import time
import requests
import uuid

app = Flask(__name__)
CORS(app)

llm_config = {"config_list": [get_openai_model_config()]}
assistant = AssistantAgent(name="assistant", llm_config=llm_config)

In [ ]:
# REPOSITORY
# Handles access to the database

def fetch_intents_from_graphdb():
    """
    Function used to get all of the intent stored in the database.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """

    # QUERY
    sparql_query = """
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT ?id ?authorName ?content ?date ?lastUpdate
    WHERE {
        ?intent a icm:Intent ;
        ex:author ?author ;
        ex:content ?content ;
        dcterms:date ?date ;
        dcterms:lastUpdateDate ?lastUpdate .
        ?author foaf:name ?authorName .
        BIND(STR(?intent) AS ?id)
    }
    ORDER BY ASC(?id)
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db",
        data={"query": sparql_query},
        headers={"Accept": "application/sparql-results+json"}
    )
    
    return response


def insert_intent_into_graphdb(intent_id : str, author : str, content : str, date : str):
    """
    Function used to insert a new intent in the database.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    author : str
        The author of the intent.
    content : str
        The content of the intent.
    date : str
        The date the intent has been made.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """

    # Cleaning of the author name
    author_without_space = author.replace(" ", "_")

    # QUERY
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    INSERT DATA {{
        ex:{intent_id} a icm:Intent ;
            ex:author <http://example.com/intent/person/{author_without_space}> ;
            ex:content "{content}" ;
            dcterms:date "{date}"^^xsd:dateTime ;
            dcterms:lastUpdateDate "{date}"^^xsd:dateTime .

        <http://example.com/intent/person/{author_without_space}> a foaf:Person ;
            foaf:name "{author}" .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query,
        headers={"Content-Type": "application/sparql-update"}
    )

    return response

def insert_report_into_graphdb(intent_report_id : str, intent_id : str, agent_name : str, agent_response : str, response_time : str):
    """
    Function used to insert a new intent report in the database.

    Parameters
    ----------
    intent_report_id : str
        The ID of the intent report.
    intent_id : str
        The ID of the intent.
    agent_name : str
        The name of the agent who handled the intent.
    agent_response : str
        The response that gave the agent.
    response_time : str
        The execution time of the handle of the intent.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """

    date = dt.datetime.now().isoformat()

    # Cleaning the  of the agent
    agent_response = agent_response.replace('\\', '\\\\')
    agent_response = agent_response.replace('"', '\\"')
    agent_response = agent_response.replace('\n', '\\n')

    # QUERY
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    INSERT DATA {{
        <http://example.com/intent/{intent_id}/intentReport/{intent_report_id}> a icm:IntentReport ;
            dcterms:date "{date}"^^xsd:dateTime ;
            ex:agentName "{agent_name}" ;
            ex:executionTime "{response_time}"^^xsd:float ;
            ex:intent <http://example.com/intent/{intent_id}> ;
            ex:response "{agent_response}" .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query.encode("utf-8"),
        headers={"Content-Type": "application/sparql-update"}
    )

    return response

def select_intent_from_graphdb(intent_id : str):
    """
    Function use to get an intent from its ID.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """
    
    # QUERY
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT ?id ?authorName ?content ?date ?lastUpdate
    WHERE {{
        BIND(<http://example.com/intent/{intent_id}> AS ?intent)
        ?intent a icm:Intent ;
        ex:author ?author ;
        ex:content ?content ;
        dcterms:date ?date ;
        dcterms:lastUpdateDate ?lastUpdate .
        ?author foaf:name ?authorName .
        BIND(STR(?intent) AS ?id)
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db",
        data=sparql_query.encode("utf-8"),
        headers={
            "Content-Type": "application/sparql-query",
            "Accept": "application/sparql-results+json"
        }
    )
    
    return response

def fetch_intent_reports_of_intent_from_graphdb(intent_id : str):
    """
    Function used to get all of the intent reports of an intent, stored in the database.
    
    Parameters
    ----------
    intent_id : str
        The ID of the intent.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """
    
    # QUERY
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>

    SELECT ?report ?date ?agentName ?response ?executionTime
    WHERE {{
        ?report a icm:IntentReport ;
                dcterms:date ?date ;
                ex:agentName ?agentName ;
                ex:response ?response ;
                ex:executionTime ?executionTime ;
                ex:intent <http://example.com/intent/{intent_id}> .
    }}
    ORDER BY ASC(?report)
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db",
        data=sparql_query.encode("utf-8"),
        headers={
            "Content-Type": "application/sparql-query",
            "Accept": "application/sparql-results+json"
        }
    )

    return response

def fetch_single_intent_report_from_graphdb(intent_id : str, report_id : str):
    """
    Function use to get an intent report of an intent, from their IDs.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    report_id : str
        The ID of the intent report.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """

    # QUERY
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>

    SELECT ?report ?date ?agentName ?response ?executionTime
    WHERE {{
        BIND(<http://example.com/intent/{intent_id}/intentReport/{report_id}> AS ?report)
        ?report a icm:IntentReport ;
            dcterms:date ?date ;
            ex:agentName ?agentName ;
            ex:response ?response ;
            ex:executionTime ?executionTime ;
            ex:intent <http://example.com/intent/{intent_id}> .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db",
        data=sparql_query.encode("utf-8"),
        headers={
            "Content-Type": "application/sparql-query",
            "Accept": "application/sparql-results+json"
        }
    )

    return response


def update_intent_content_from_graphdb(intent_id : str, new_content : str):
    """
    Function used to update the content of an intent from its ID.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    new_content : str
        The new content of the intent.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """

    updateDate = dt.datetime.now().isoformat()

    # QUERY
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    
    DELETE {{
        <http://example.com/intent/{intent_id}> ex:content ?oldContent .
        <http://example.com/intent/{intent_id}> dcterms:lastUpdateDate ?oldDate .
    }}
    INSERT {{
        <http://example.com/intent/{intent_id}> ex:content "{new_content}" .
        <http://example.com/intent/{intent_id}> dcterms:lastUpdateDate "{updateDate}"^^xsd:dateTime .
    }}
    WHERE {{
        <http://example.com/intent/{intent_id}> ex:content ?oldContent .
        OPTIONAL {{ <http://example.com/intent/{intent_id}> dcterms:lastUpdateDate ?oldDate . }}
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query.encode("utf-8"),
        headers={"Content-Type": "application/sparql-update"}
    )

    return response


def delete_intent_from_graphdb(intent_id : str):
    """
    Function used to delete an intent from its ID.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """

    # QUERY
    sparql_query = f"""
    DELETE {{
        <http://example.com/intent/{intent_id}> ?p ?o .
    }}
    WHERE {{
        <http://example.com/intent/{intent_id}> ?p ?o .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query.encode("utf-8"),
        headers={"Content-Type": "application/sparql-update"}
    )
    
    return response

def delete_reports_of_intent_from_graphdb(intent_id : str):
    """
    Function used to delete all intent reports of an intent.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """

    # QUERY
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>

    DELETE {{
        ?report ?p ?o .
    }}
    WHERE {{
        ?report ex:intent <http://example.com/intent/{intent_id}> .
        ?report ?p ?o .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query.encode("utf-8"),
        headers={"Content-Type": "application/sparql-update"}
    )
    
    return response

def delete_report_of_intent_from_graphdb(intent_id : str, report_id : str):
    """
    Function used to delete an intent report of an intent, from their IDs.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    report_id : str
        The ID of the intent report.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """
    
    # QUERY
    sparql_query = f"""
    DELETE {{
        <http://example.com/intent/{intent_id}/intentReport/{report_id}> ?p ?o .
    }}
    WHERE {{
        <http://example.com/intent/{intent_id}/intentReport/{report_id}> ?p ?o .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query.encode("utf-8"),
        headers={"Content-Type": "application/sparql-update"}
    )

    return response


In [ ]:
# SERVICE
# Contains logic

def get_all_intents():
    """
    Function used to analyse the response of the database to get all the intents stored and
    to format them.
    
    Returns
    -------
    intents : List
        List of intents.
    """

    response = fetch_intents_from_graphdb()

    if response.status_code != 200:
        return None
    
    intents_db = response.json()["results"]["bindings"]

    if not intents_db:
        return None

    intents = []

    for i in intents_db:
        id = i['id']["value"].split("/")[-1]

        intent = {
            "id": id,
            "author": i["authorName"]["value"],
            "content": i["content"]["value"],
            "date": i["date"]["value"],
            "lastUpdateDate": i["lastUpdate"]["value"]
        }
        intents.append(intent)
    
    return intents

def get_all_intents_json_ld():
    """
    Function used to format the list of intent in JSON-LD format.
    
    Returns
    -------
    jsonld_data : List
        List of intents in JSON-LD format.
    """
    
    intents = get_all_intents()
    
    if not intents:
        return None
    
    # Definition of the context
    jsonld_data = {
        "@context": {
            "ex": "http://example.com/intent/",
            "icm": "http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/",
            "foaf": "http://xmlns.com/foaf/0.1/",
            "dcterms": "http://purl.org/dc/terms/",
            "xsd": "http://www.w3.org/2001/XMLSchema# "
        }
    }
    jsonld_data["intents"] = []

    for intent in intents:
        id = intent['id'].split("/")[-1]
        
        date_obj = dt.datetime.fromisoformat(intent['date'])
        update_obj = dt.datetime.fromisoformat(intent['lastUpdateDate'])

        intent_data = {
            "@id": f"ex:{id}",
            "@type": "icm:Intent",
            "ex:author": {
                "@id": f"ex:person/{intent['author'].replace(' ', '_')}",
                "@type": "foaf:Person",
                "foaf:name": intent['author']
            },
            "ex:content": intent['content'],
            "dcterms:date": {
                "@value": date_obj.isoformat(),
                "@type": "xsd:dateTime"
            },
            "dcterms:lastUpdateDate": {
                "@value": update_obj.isoformat(),
                "@type": "xsd:dateTime"
            }
        }
        jsonld_data["intents"].append(intent_data)

    return jsonld_data

def select_intent_from_id(intent_id : str):
    """
    Function used to analyse the response of the database to get an intent from its ID and
    to format it.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    
    Returns
    -------
    intent : Dictionary
        The intent with all its attributes.
    """
    
    response = select_intent_from_graphdb(intent_id)

    if response.status_code != 200:
        return None

    intent_db = response.json()["results"]["bindings"][0]

    if not intent_db:
        return None

    id = intent_db['id']["value"].split("/")[-1]

    intent = {
        "id": id,
        "author": intent_db["authorName"]["value"],
        "content": intent_db["content"]["value"],
        "date": intent_db["date"]["value"],
        "lastUpdateDate": intent_db["lastUpdate"]["value"]
    }
    
    return intent

def select_intent_json_ld_from_id(intent_id : str):
    """
    Function used to format an intent in JSON-LD format.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    
    Returns
    -------
    jsonld_data : Dictionary
        The intent with all its attributes in JSON-LD format.
    """
    
    response = select_intent_from_graphdb(intent_id)

    if response.status_code != 200:
        return None

    intent_db = response.json()["results"]["bindings"][0]

    if not intent_db:
        return None

    id = intent_db['id']["value"].split("/")[-1]
    
    date_obj = dt.datetime.fromisoformat(intent_db["date"]["value"])
    update_obj = dt.datetime.fromisoformat(intent_db["lastUpdate"]["value"])

    jsonld_data = {
        # Definition of the context
        "@context": {
            "ex": "http://example.com/intent/",
            "icm": "http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/",
            "foaf": "http://xmlns.com/foaf/0.1/",
            "dcterms": "http://purl.org/dc/terms/",
            "xsd": "http://www.w3.org/2001/XMLSchema# "
        },
        "@id": f"ex:{id}",
        "@type": "icm:Intent",
        "ex:author": {
            "@id": f"ex:person/{intent_db["authorName"]["value"].replace(' ', '_')}",
            "@type": "foaf:Person",
            "foaf:name": intent_db["authorName"]["value"]
        },
        "ex:content": intent_db["content"]["value"],
        "dcterms:date": {
            "@value": date_obj.isoformat(),
            "@type": "xsd:dateTime"
        },
        "dcterms:lastUpdateDate": {
            "@value": update_obj.isoformat(),
            "@type": "xsd:dateTime"
        }
    }
    
    return jsonld_data

def get_all_intent_reports_of_intent(intent_id : str):
    """
    Function used to analyse the response of the database to get all the intent reports 
    of an intent from its ID and to format them.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    
    Returns
    -------
    intent_reports : List
        The list of all of the intent reports and their attributes.
    """
    
    if not select_intent_from_id(intent_id):
        return None
    
    response = fetch_intent_reports_of_intent_from_graphdb(intent_id)

    if response.status_code != 200:
        return None

    results = response.json()["results"]["bindings"]

    if not results:
        return None

    intent_reports = []

    for r in results:
        id = r['report']["value"].split("/")[-1]

        intent_report = {
            "id": id,
            "agentName": r["agentName"]["value"],
            "response": r["response"]["value"],
            "date": r["date"]["value"],
            "executionTime": r["executionTime"]["value"]
        }
        intent_reports.append(intent_report)
    
    return intent_reports
    
def get_intent_report_of_intent(intent_id : str, report_id : str):
    """
    Function used to analyse the response of the database to get an intent report
    of an intent from their IDs and to format it.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    report_id : str
        The ID of the intent report.
    
    Returns
    -------
    intent_report : Dictionnary
        The list of all of the intent reports and their attributes.
    """
    
    if not select_intent_from_id(intent_id):
        return None
    
    response = fetch_single_intent_report_from_graphdb(intent_id, report_id)

    if response.status_code != 200:
        return None

    result = response.json()["results"]["bindings"]

    if not result:
        return None

    intent_report_data = result[0]

    id = result[0]['report']["value"].split("/")[-1]

    intent_report = {
        "id": id,
        "agentName": intent_report_data["agentName"]["value"],
        "response": intent_report_data["response"]["value"],
        "date": intent_report_data["date"]["value"],
        "executionTime": intent_report_data["executionTime"]["value"]
    }
    
    return intent_report


def create_intent(author : str, content : str):
    """
    Function used to 

    Parameters
    ----------
    author : str
        The author of the intent.
    content : str
        The content of the intent.
    
    Returns
    -------
    int
        DESCRIPTION.
    """
    
    intent_id = uuid.uuid4()
    date = dt.datetime.now().isoformat()
    
    response = insert_intent_into_graphdb(intent_id, author, content, date)
    
    if response.status_code not in {200, 201, 202, 204}:
        return 
    
    response = create_intent_report(intent_id, content)

    if response.status_code not in {200, 201, 202, 204}:
        return
    
    # TODO return intent and report
    # {"intent":intent.to_dict(), "report":report.to_dict()}
    return 201

def create_intent_report(intent_id : str, content : str):
    # Handle of the intent

    startTime = time.time()
    
    # TODO

#   prompt = f"""
#   You are an assistant designed to convert user intents into actionable instructions.
#
#   Given the following user intent:
#   \"{content}\"
#
#   Your task is to:
#   1. Understand the intent clearly.
#   2. Break it down into concrete actions.
#   3. Return a concise action plan in plain English.
#
#   Output format:
#   - Step-by-step list of instructions
#   - Clear and unambiguous
#   """
#
#   agent_response = assistant.generate_reply(
#       messages=[{"role": "user", "content": prompt}]
#   )
    
    #print(agent_response)

    agent_response = "reply"

    # Cleaning of the response

    if(agent_response.endswith("TERMINATE")):
        agent_response = agent_response.replace("TERMINATE", "").strip()

    report_id = uuid.uuid4()
    
    response = insert_report_into_graphdb(report_id, intent_id, assistant.name, agent_response, (time.time() - startTime))

    return response

def update_partially_intent(intent_id : str, content : str):
    if(select_intent_from_id(intent_id)):
        update_intent_content_from_graphdb(intent_id, content)
        response = create_intent_report(intent_id, content)
        return response
    else:
        return None

def delete_intent_by_id(intent_id : str):
    response = delete_reports_of_intent_from_graphdb(intent_id)
    return delete_intent_from_graphdb(intent_id)

def delete_report_by_id(intent_id : str, report_id : str):
    return delete_report_of_intent_from_graphdb(intent_id, report_id)
    


In [ ]:
# CONTROLLER
# Handles HTTP requests

# GET METHODS

# Retrieves the list of intent objects

@app.get("/intent")
def get_intents():
    try:
        intents = get_all_intents()
        if intents:
            return jsonify(intents), 200
        else :
            return '', 204
    except Exception as e:
        return str(e), 500

# Retrieves the list of intent and return it as RDF in JSON-LD format

@app.get("/intent/json-ld")
def get_intents_json_ld():
    try:
        intents = get_all_intents_json_ld()
        if intents:
            return jsonify(intents), 200
        else :
            return '', 204
    except Exception as e:
        return str(e), 500
    
# Retrieves an Intent by ID

@app.get("/intent/<string:intent_id>")
def get_intent_from_id(intent_id : str):
    try:
        intent = select_intent_from_id(intent_id)
        if intent:
            return jsonify(intent), 200
        else :
            return '', 204
    except Exception as e:
        return str(e), 500

# Retrieves an Intent by ID and return it as RDF in JSON-LD format

@app.get("/intent/<string:intent_id>/json-ld")
def get_intent_json_ld_from_id(intent_id : str):
    try:
        intent = select_intent_json_ld_from_id(intent_id)
        if intent:
            return jsonify(intent), 200
        else :
            return '', 204
    except Exception as e:
        return str(e), 500
    

# Intent Report

# Retrieves the list of IntentReport of an Intent

@app.get("/intent/<string:intent_id>/intentReport")
def get_intent_report_of_intent_id(intent_id : str):
    try:
        intent_reports = get_all_intent_reports_of_intent(intent_id)
        if intent_reports:
            return jsonify(intent_reports), 200
        else :
            return '', 204
    except Exception as e:
        return str(e), 500

# Retrieves an IntentReport of an Intent

@app.get("/intent/<string:intent_id>/intentReport/<string:report_id>")
def get_intent_report_from_id_of_intent_id(intent_id : str, report_id : str):
    try:
        intent_report = get_intent_report_of_intent(intent_id, report_id)
        if intent_report:
            return jsonify(intent_report), 200
        else :
            return '', 204
    except Exception as e:
        return str(e), 500

# POST METHODS

# Creates an Intent

@app.post("/intent")
def add_intent():
    if request.is_json:
       added_intent = request.get_json()
       
       create_intent(added_intent["author"], added_intent["content"])

       return {"message": "Intent created"}, 201
    else :
        return {"error": "Request must be JSON"}, 415
    

# PATCH METHODS

# Partially updates an Intent

@app.patch("/intent/<string:intent_id>")
def update_intent(intent_id : str):
    updated_intent = request.get_json()

    if "content" in updated_intent and updated_intent["content"] != "":
        response = update_partially_intent(intent_id, updated_intent["content"])

        if(response):
            return get_intent_from_id(intent_id)
        else :
            return {"error": "No intent with id " + str(intent_id)}, 404
        

# DELETE METHODS

# Deletes an intent

@app.delete("/intent/<string:intent_id>")
def delete_intent(intent_id : str):
    # TODO check if intent exists
    response = delete_intent_by_id(intent_id)
    if response.status_code == 204:
        return jsonify({"message": "Intent deleted successfully"}), 200
    else:
        return jsonify({"error": response.text}), 500


# Deletes an IntentReport of an Intent

@app.delete("/intent/<string:intent_id>/intentReport/<string:report_id>")
def delete_intent_report_from_intent(intent_id : str, report_id : str):
    # TODO check if intent and report exist
    response = delete_report_by_id(intent_id, report_id)
    if response.status_code == 204:
        return jsonify({"message": "IntentReport deleted successfully"}), 200
    else:
        return jsonify({"error": response.text}), 500



In [ ]:
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=False)